<a href="https://colab.research.google.com/github/hfwalvir/Dissertation/blob/main/Homomorphic_Encryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install tenseal

In [30]:
import pandas as pd
import random

# File path - adjust this to match your system
file_path = '/content/sample_data/combined_financial_data_idx.csv'

# Read the CSV file
data = pd.read_csv(file_path)

In [31]:
data.head()



,symbol,account,type,2020,2021,2022,2023
0,AALI,Accounts Payable,BS,7.702640e+11,1.026717e+12,1.224423e+12,8.420640e+11
1,AALI,Accounts Receivable,BS,7.658490e+11,4.581350e+11,8.487700e+11,6.744870e+11
2,AALI,Accumulated Depreciation,BS,-1.092095e+13,-1.213381e+13,-1.330375e+13,-1.443685e+13
3,AALI,Additional Paid In Capital,BS,3.878995e+12,3.878995e+12,3.878995e+12,3.878995e+12
4,AALI,Allowance For Doubtful Accounts Receivable,BS,-2.426100e+10,-2.454300e+10,-2.705700e+10,-2.651600e+10


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89243 entries, 0 to 89242
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   symbol   89243 non-null  object 
 1   account  89243 non-null  object 
 2   type     89243 non-null  object 
 3   2020     83988 non-null  float64
 4   2021     84474 non-null  float64
 5   2022     83634 non-null  float64
 6   2023     69831 non-null  float64
dtypes: float64(4), object(3)
memory usage: 4.8+ MB


# **ACCOUNTING EQUATION CALCULATION (VERIFICATION)**

In [33]:
import pandas as pd
import tenseal as ts

# Set up the TenSEAL context
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.generate_galois_keys()
context.global_scale = 2**40

# Filter data for AALI and the year 2023
aali_2023 = data[(data['symbol'] == 'AALI') & (data['2023'].notna())]

# Extract financial data
financial_data = {
    'current_assets': int(float(aali_2023[aali_2023['account'] == 'Current Assets']['2023'].values[0])),
    'non_current_assets': int(float(aali_2023[aali_2023['account'] == 'Total Assets']['2023'].values[0])) -
                          int(float(aali_2023[aali_2023['account'] == 'Current Assets']['2023'].values[0])),
    'current_liabilities': int(float(aali_2023[aali_2023['account'] == 'Current Liabilities']['2023'].values[0])),
    'non_current_liabilities': int(float(aali_2023[aali_2023['account'] == 'Total Liabilities Net Minority Interest']['2023'].values[0])) -
                               int(float(aali_2023[aali_2023['account'] == 'Current Liabilities']['2023'].values[0])),
    'total_equity': int(float(aali_2023[aali_2023['account'] == 'Total Equity Gross Minority Interest']['2023'].values[0]))
}

# Encrypt financial data
encrypted_data = {key: ts.ckks_vector(context, [value]) for key, value in financial_data.items()}

# Calculate total assets and total liabilities + equity
total_assets = encrypted_data['current_assets'] + encrypted_data['non_current_assets']
total_liabilities_and_equity = encrypted_data['current_liabilities'] + encrypted_data['non_current_liabilities'] + encrypted_data['total_equity']

# Calculate the difference (should be close to zero if balanced)
balance_difference = total_assets - total_liabilities_and_equity

# Function to decrypt and print results
def decrypt_and_print(name, encrypted_value):
    decrypted_value = encrypted_value.decrypt()[0]
    print(f"{name}: {decrypted_value:.2f}")

print("Balance Sheet Components:")
decrypt_and_print("Total Assets", total_assets)
decrypt_and_print("Total Liabilities and Equity", total_liabilities_and_equity)
print("\nBalance Check:")
decrypt_and_print("Difference", balance_difference)

# Print original financial data for verification
print("\nOriginal Financial Data:")
for key, value in financial_data.items():
    print(f"{key}: {value}")

Balance Sheet Components:
Total Assets: 28846243000000.00
Total Liabilities and Equity: 28846243000000.00

Balance Check:
Difference: -0.00

Original Financial Data:
current_assets: 7118202000000
non_current_assets: 21728041000000
current_liabilities: 3882141000000
non_current_liabilities: 2398096000000
total_equity: 22566006000000


# **Decrypting both numerator and denominator For Ratio Analysis of one company- AALI**

In [34]:
import pandas as pd
import tenseal as ts

# Set up the TenSEAL context
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.generate_galois_keys()
context.global_scale = 2**40

# Filter data for AALI and the year 2023
aali_2023 = data[(data['symbol'] == 'AALI') & (data['2023'].notna())]

# Extract financial data
financial_data = {
    'current_assets': int(float(aali_2023[aali_2023['account'] == 'Current Assets']['2023'].values[0])),
    'current_liabilities': int(float(aali_2023[aali_2023['account'] == 'Current Liabilities']['2023'].values[0])),
    'total_liabilities': int(float(aali_2023[aali_2023['account'] == 'Total Liabilities Net Minority Interest']['2023'].values[0])),
    'total_equity': int(float(aali_2023[aali_2023['account'] == 'Total Equity Gross Minority Interest']['2023'].values[0])),
    'total_assets': int(float(aali_2023[aali_2023['account'] == 'Total Assets']['2023'].values[0])),
    'net_income': int(float(aali_2023[aali_2023['account'] == 'Net Income']['2023'].values[0])),
    'total_revenue': int(float(aali_2023[aali_2023['account'] == 'Operating Revenue']['2023'].values[0]))
}

# Encrypt financial data
encrypted_data = {key: ts.ckks_vector(context, [value]) for key, value in financial_data.items()}

# Helper function for division
def encrypted_division(numerator, denominator):
    # Decrypt both numerator and denominator
    num_value = numerator.decrypt()[0]
    den_value = denominator.decrypt()[0]
    # Perform division
    result = num_value / den_value
    # Re-encrypt the result
    return ts.ckks_vector(context, [result])

# Calculate financial ratios
current_ratio = encrypted_division(encrypted_data['current_assets'], encrypted_data['current_liabilities'])
debt_to_equity = encrypted_division(encrypted_data['total_liabilities'], encrypted_data['total_equity'])
roa = encrypted_division(encrypted_data['net_income'], encrypted_data['total_assets'])
profit_margin = encrypted_division(encrypted_data['net_income'], encrypted_data['total_revenue'])

print("Encrypted Financial Ratios:")
print("Current Ratio: Encrypted")
print("Debt-to-Equity Ratio: Encrypted")
print("Return on Assets (ROA): Encrypted")
print("Profit Margin: Encrypted")

# Function to decrypt and print results
def decrypt_and_print(name, encrypted_value):
    decrypted_value = encrypted_value.decrypt()[0]
    print(f"{name}: {decrypted_value:.4f}")

print("\nDecrypted Results:")
decrypt_and_print("Current Ratio", current_ratio)
decrypt_and_print("Debt-to-Equity Ratio", debt_to_equity)
decrypt_and_print("Return on Assets (ROA)", roa)
decrypt_and_print("Profit Margin", profit_margin)

# Print original financial data for verification
print("\nOriginal Financial Data:")
for key, value in financial_data.items():
    print(f"{key}: {value}")

Encrypted Financial Ratios:
Current Ratio: Encrypted
Debt-to-Equity Ratio: Encrypted
Return on Assets (ROA): Encrypted
Profit Margin: Encrypted

Decrypted Results:
Current Ratio: 1.8336
Debt-to-Equity Ratio: 0.2783
Return on Assets (ROA): 0.0366
Profit Margin: 0.0509

Original Financial Data:
current_assets: 7118202000000
current_liabilities: 3882141000000
total_liabilities: 6280237000000
total_equity: 22566006000000
total_assets: 28846243000000
net_income: 1055897000000
total_revenue: 20745473000000


# **Encrypting numerator and decrypting denominator For Ratio Analysis of one company- AALI**

In [35]:
import pandas as pd
import tenseal as ts

# Set up the TenSEAL context
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.generate_galois_keys()
context.global_scale = 2**40

# Filter data for AALI and the year 2023
aali_2023 = data[(data['symbol'] == 'AALI') & (data['2023'].notna())]

# Extract financial data
financial_data = {
    'current_assets': int(float(aali_2023[aali_2023['account'] == 'Current Assets']['2023'].values[0])),
    'current_liabilities': int(float(aali_2023[aali_2023['account'] == 'Current Liabilities']['2023'].values[0])),
    'total_liabilities': int(float(aali_2023[aali_2023['account'] == 'Total Liabilities Net Minority Interest']['2023'].values[0])),
    'total_equity': int(float(aali_2023[aali_2023['account'] == 'Total Equity Gross Minority Interest']['2023'].values[0])),
    'total_assets': int(float(aali_2023[aali_2023['account'] == 'Total Assets']['2023'].values[0])),
    'net_income': int(float(aali_2023[aali_2023['account'] == 'Net Income']['2023'].values[0])),
    'total_revenue': int(float(aali_2023[aali_2023['account'] == 'Operating Revenue']['2023'].values[0]))
}

# Encrypt financial data
encrypted_data = {key: ts.ckks_vector(context, [value]) for key, value in financial_data.items()}

# Helper function for division
def encrypted_division(numerator, denominator):
    # Decrypt only the denominator
    den_value = denominator.decrypt()[0]
    # Compute the inverse of the denominator
    inverse = 1 / den_value
    # Encrypt the inverse
    encrypted_inverse = ts.ckks_vector(context, [inverse])
    # Multiply the numerator by the encrypted inverse
    return numerator * encrypted_inverse

# Calculate financial ratios
current_ratio = encrypted_division(encrypted_data['current_assets'], encrypted_data['current_liabilities'])
debt_to_equity = encrypted_division(encrypted_data['total_liabilities'], encrypted_data['total_equity'])
roa = encrypted_division(encrypted_data['net_income'], encrypted_data['total_assets'])
profit_margin = encrypted_division(encrypted_data['net_income'], encrypted_data['total_revenue'])

print("Encrypted Financial Ratios:")
print("Current Ratio: Encrypted")
print("Debt-to-Equity Ratio: Encrypted")
print("Return on Assets (ROA): Encrypted")
print("Profit Margin: Encrypted")

# Function to decrypt and print results (for demonstration purposes only)
def decrypt_and_print(name, encrypted_value):
    decrypted_value = encrypted_value.decrypt()[0]
    print(f"{name}: {decrypted_value:.4f}")

print("\nDecrypted Results ")
decrypt_and_print("Current Ratio", current_ratio)
decrypt_and_print("Debt-to-Equity Ratio", debt_to_equity)
decrypt_and_print("Return on Assets (ROA)", roa)
decrypt_and_print("Profit Margin", profit_margin)

# Print original financial data for verification
print("\nOriginal Financial Data:")
for key, value in financial_data.items():
    print(f"{key}: {value}")

Encrypted Financial Ratios:
Current Ratio: Encrypted
Debt-to-Equity Ratio: Encrypted
Return on Assets (ROA): Encrypted
Profit Margin: Encrypted

Decrypted Results 
Current Ratio: 1248.1594
Debt-to-Equity Ratio: -325.7400
Return on Assets (ROA): -23.1453
Profit Margin: -197.7832

Original Financial Data:
current_assets: 7118202000000
current_liabilities: 3882141000000
total_liabilities: 6280237000000
total_equity: 22566006000000
total_assets: 28846243000000
net_income: 1055897000000
total_revenue: 20745473000000


# **TRYING HOMOMORPHIC ENCRYPTION WITH NEW COMPANY ADDITION : AALI AND ABDA**

In [36]:
data = pd.read_csv('/content/sample_data/combined_financial_data_idx.csv')

In [37]:
import pandas as pd
import tenseal as ts

# Set up the TenSEAL context
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.generate_galois_keys()
context.global_scale = 2**40


# Function to extract and encrypt balance sheet data for a company
def extract_and_encrypt_balance_sheet(symbol):
    company_data = data[(data['symbol'] == symbol) & (data['2023'].notna())]
    balance_sheet_data = {
        'total_assets': int(float(company_data[company_data['account'] == 'Total Assets']['2023'].values[0])),
        'total_liabilities': int(float(company_data[company_data['account'] == 'Total Liabilities Net Minority Interest']['2023'].values[0])),
        'total_equity': int(float(company_data[company_data['account'] == 'Total Equity Gross Minority Interest']['2023'].values[0]))
    }
    return {key: ts.ckks_vector(context, [value]) for key, value in balance_sheet_data.items()}

# Extract and encrypt data for AALI and ABDA
encrypted_aali = extract_and_encrypt_balance_sheet('AALI')
encrypted_abda = extract_and_encrypt_balance_sheet('ABDA')

# Function to check balance sheet equation
def check_balance_sheet_equation(data):
    assets = data['total_assets']
    liabilities_plus_equity = data['total_liabilities'] + data['total_equity']
    difference = assets - liabilities_plus_equity
    return assets, liabilities_plus_equity, difference

# Check balance sheet equation for both companies
aali_results = check_balance_sheet_equation(encrypted_aali)
abda_results = check_balance_sheet_equation(encrypted_abda)

# Function to decrypt and print results
def decrypt_and_print_balance_sheet(company_name, results):
    assets, liabilities_plus_equity, difference = results
    print(f"\nBalance Sheet Equation Check for {company_name}:")
    print(f"Total Assets: {assets.decrypt()[0]:.2f}")
    print(f"Total Liabilities + Equity: {liabilities_plus_equity.decrypt()[0]:.2f}")
    print(f"Difference: {difference.decrypt()[0]:.2f}")

# Print results
decrypt_and_print_balance_sheet("AALI", aali_results)
decrypt_and_print_balance_sheet("ABDA", abda_results)

# Printing original data for verification
def print_original_data(symbol):
    company_data = data[(data['symbol'] == symbol) & (data['2023'].notna())]
    print(f"\nOriginal Balance Sheet Data for {symbol}:")
    for account in ['Total Assets', 'Total Liabilities Net Minority Interest', 'Total Equity Gross Minority Interest']:
        value = int(float(company_data[company_data['account'] == account]['2023'].values[0]))
        print(f"{account}: {value}")

print_original_data('AALI')
print_original_data('ABDA')


Balance Sheet Equation Check for AALI:
Total Assets: 28846243000000.00
Total Liabilities + Equity: 28846243000000.00
Difference: -0.00

Balance Sheet Equation Check for ABDA:
Total Assets: 2664450707000.00
Total Liabilities + Equity: 2664450707000.00
Difference: -0.00

Original Balance Sheet Data for AALI:
Total Assets: 28846243000000
Total Liabilities Net Minority Interest: 6280237000000
Total Equity Gross Minority Interest: 22566006000000

Original Balance Sheet Data for ABDA:
Total Assets: 2664450707000
Total Liabilities Net Minority Interest: 1115590529000
Total Equity Gross Minority Interest: 1548860178000


# **TRYING HOMOMORPHIC ENCRYPTION FOR RATIO ANALYSIS WITH 4 COMPANIES, WITH NUMERATOR ENCRYPTED AND DENOMINATOR DECRYPTED : AALI, ABMM,ACES,ACST**



In [38]:
data = pd.read_csv('/content/sample_data/combined_financial_data_idx.csv')

In [39]:
import pandas as pd
import tenseal as ts

# Set up the TenSEAL context
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.generate_galois_keys()
context.global_scale = 2**40


# Function to extract and encrypt financial data for a company
def extract_and_encrypt_data(symbol):
    company_data = data[(data['symbol'] == symbol) & (data['2023'].notna())]
    financial_data = {
        'current_assets': float(company_data[company_data['account'] == 'Current Assets']['2023'].values[0]),
        'current_liabilities': float(company_data[company_data['account'] == 'Current Liabilities']['2023'].values[0]),
        'total_liabilities': float(company_data[company_data['account'] == 'Total Liabilities Net Minority Interest']['2023'].values[0]),
        'total_equity': float(company_data[company_data['account'] == 'Total Equity Gross Minority Interest']['2023'].values[0]),
        'total_assets': float(company_data[company_data['account'] == 'Total Assets']['2023'].values[0]),
        'net_income': float(company_data[company_data['account'] == 'Net Income']['2023'].values[0]),
        'operating_revenue': float(company_data[company_data['account'] == 'Operating Revenue']['2023'].values[0])
    }
    return {key: ts.ckks_vector(context, [value]) for key, value in financial_data.items()}

# List of companies
companies = ['AALI', 'ABMM', 'ACES', 'ACST']

# Extract and encrypt data for all companies
encrypted_data = {company: extract_and_encrypt_data(company) for company in companies}

# Helper function for division
def encrypted_division(numerator, denominator):
    den_value = denominator.decrypt()[0]
    inverse = 1 / den_value
    encrypted_inverse = ts.ckks_vector(context, [inverse])
    return numerator * encrypted_inverse

# Function to calculate financial ratios
def calculate_ratios(data):
    current_ratio = encrypted_division(data['current_assets'], data['current_liabilities'])
    debt_to_equity = encrypted_division(data['total_liabilities'], data['total_equity'])
    roa = encrypted_division(data['net_income'], data['total_assets'])
    profit_margin = encrypted_division(data['net_income'], data['operating_revenue'])
    return current_ratio, debt_to_equity, roa, profit_margin

# Calculate ratios for all companies
company_ratios = {company: calculate_ratios(data) for company, data in encrypted_data.items()}

# Function to decrypt and print results
def decrypt_and_print_ratios(company_name, ratios):
    current_ratio, debt_to_equity, roa, profit_margin = ratios
    print(f"\nFinancial Ratios for {company_name}:")
    print(f"Current Ratio: {current_ratio.decrypt()[0]:.2f}")
    print(f"Debt-to-Equity Ratio: {debt_to_equity.decrypt()[0]:.2f}")
    print(f"Return on Assets (ROA): {roa.decrypt()[0]:.4f}")
    print(f"Profit Margin: {profit_margin.decrypt()[0]:.4f}")

# Print results for all companies
for company, ratios in company_ratios.items():
    decrypt_and_print_ratios(company, ratios)

# Print original data for verification (optional)
def print_original_data(symbol):
    company_data = data[(data['symbol'] == symbol) & (data['2023'].notna())]
    print(f"\nOriginal Financial Data for {symbol}:")
    for account in ['Current Assets', 'Current Liabilities', 'Total Liabilities Net Minority Interest',
                    'Total Equity Gross Minority Interest', 'Total Assets', 'Net Income', 'Operating Revenue']:
        value = float(company_data[company_data['account'] == account]['2023'].values[0])
        print(f"{account}: {value}")

for company in companies:
    print_original_data(company)


Financial Ratios for AALI:
Current Ratio: -11859.00
Debt-to-Equity Ratio: 6557.76
Return on Assets (ROA): 2302.3041
Profit Margin: 317.5040

Financial Ratios for ABMM:
Current Ratio: 1.52
Debt-to-Equity Ratio: -2.29
Return on Assets (ROA): 0.4486
Profit Margin: 0.7363

Financial Ratios for ACES:
Current Ratio: -2380.48
Debt-to-Equity Ratio: -1495.33
Return on Assets (ROA): 280.7520
Profit Margin: 221.8893

Financial Ratios for ACST:
Current Ratio: 866.89
Debt-to-Equity Ratio: -2213.46
Return on Assets (ROA): -165.0667
Profit Margin: -536.8814

Original Financial Data for AALI:
Current Assets: 7118202000000.0
Current Liabilities: 3882141000000.0
Total Liabilities Net Minority Interest: 6280237000000.0
Total Equity Gross Minority Interest: 22566006000000.0
Total Assets: 28846243000000.0
Net Income: 1055897000000.0
Operating Revenue: 20745473000000.0

Original Financial Data for ABMM:
Current Assets: 622722099.0
Current Liabilities: 631680763.0
Total Liabilities Net Minority Interest: 13

# **TRYING HOMOMORPHIC ENCRYPTION FOR RATIO ANALYSIS WITH 4 COMPANIES, WITH NUMERATOR DECRYPTED AND DENOMINATOR DECRYPTED : AALI, ABMM,ACES,ACST**

In [40]:
data = pd.read_csv('/content/sample_data/combined_financial_data_idx.csv')

In [41]:
import pandas as pd
import tenseal as ts

# Set up the TenSEAL context
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.generate_galois_keys()
context.global_scale = 2**40

# Function to extract and encrypt financial data for a company
def extract_and_encrypt_data(symbol):
    company_data = data[(data['symbol'] == symbol) & (data['2023'].notna())]
    financial_data = {
        'current_assets': float(company_data[company_data['account'] == 'Current Assets']['2023'].values[0]),
        'current_liabilities': float(company_data[company_data['account'] == 'Current Liabilities']['2023'].values[0]),
        'total_liabilities': float(company_data[company_data['account'] == 'Total Liabilities Net Minority Interest']['2023'].values[0]),
        'total_equity': float(company_data[company_data['account'] == 'Total Equity Gross Minority Interest']['2023'].values[0]),
        'total_assets': float(company_data[company_data['account'] == 'Total Assets']['2023'].values[0]),
        'net_income': float(company_data[company_data['account'] == 'Net Income']['2023'].values[0]),
        'operating_revenue': float(company_data[company_data['account'] == 'Operating Revenue']['2023'].values[0])
    }
    return {key: ts.ckks_vector(context, [value]) for key, value in financial_data.items()}

# List of companies
companies = ['AALI', 'ABMM', 'ACES', 'ACST']

# Extract and encrypt data for all companies
encrypted_data = {company: extract_and_encrypt_data(company) for company in companies}

# Helper function for division (now re-encrypts the result)
def encrypted_division(numerator, denominator):
    num_value = numerator.decrypt()[0]
    den_value = denominator.decrypt()[0]
    result = num_value / den_value if den_value != 0 else float('inf')
    return ts.ckks_vector(context, [result])

# Function to calculate financial ratios
def calculate_ratios(data):
    current_ratio = encrypted_division(data['current_assets'], data['current_liabilities'])
    debt_to_equity = encrypted_division(data['total_liabilities'], data['total_equity'])
    roa = encrypted_division(data['net_income'], data['total_assets'])
    profit_margin = encrypted_division(data['net_income'], data['operating_revenue'])
    return current_ratio, debt_to_equity, roa, profit_margin

# Calculate ratios for all companies
company_ratios = {company: calculate_ratios(data) for company, data in encrypted_data.items()}

# Function to decrypt and print results
def decrypt_and_print_ratios(company_name, ratios):
    current_ratio, debt_to_equity, roa, profit_margin = ratios
    print(f"\nFinancial Ratios for {company_name}:")
    print(f"Current Ratio: {current_ratio.decrypt()[0]:.2f}")
    print(f"Debt-to-Equity Ratio: {debt_to_equity.decrypt()[0]:.2f}")
    print(f"Return on Assets (ROA): {roa.decrypt()[0]:.4f}")
    print(f"Profit Margin: {profit_margin.decrypt()[0]:.4f}")

# Print results for all companies
for company, ratios in company_ratios.items():
    decrypt_and_print_ratios(company, ratios)

# Print original data for verification (optional)
def print_original_data(symbol):
    company_data = data[(data['symbol'] == symbol) & (data['2023'].notna())]
    print(f"\nOriginal Financial Data for {symbol}:")
    for account in ['Current Assets', 'Current Liabilities', 'Total Liabilities Net Minority Interest',
                    'Total Equity Gross Minority Interest', 'Total Assets', 'Net Income', 'Operating Revenue']:
        value = float(company_data[company_data['account'] == account]['2023'].values[0])
        print(f"{account}: {value}")

for company in companies:
    print_original_data(company)


Financial Ratios for AALI:
Current Ratio: 1.83
Debt-to-Equity Ratio: 0.28
Return on Assets (ROA): 0.0366
Profit Margin: 0.0509

Financial Ratios for ABMM:
Current Ratio: 0.99
Debt-to-Equity Ratio: 1.84
Return on Assets (ROA): 0.1340
Profit Margin: 0.1936

Financial Ratios for ACES:
Current Ratio: 7.42
Debt-to-Equity Ratio: 0.25
Return on Assets (ROA): 0.0985
Profit Margin: 0.1003

Financial Ratios for ACST:
Current Ratio: 1.00
Debt-to-Equity Ratio: 5.59
Return on Assets (ROA): -0.1036
Profit Margin: -0.1150

Original Financial Data for AALI:
Current Assets: 7118202000000.0
Current Liabilities: 3882141000000.0
Total Liabilities Net Minority Interest: 6280237000000.0
Total Equity Gross Minority Interest: 22566006000000.0
Total Assets: 28846243000000.0
Net Income: 1055897000000.0
Operating Revenue: 20745473000000.0

Original Financial Data for ABMM:
Current Assets: 622722099.0
Current Liabilities: 631680763.0
Total Liabilities Net Minority Interest: 1397760928.0
Total Equity Gross Minori

# **HOMOMORPHIC ENCRYPTION WITH NUMERATOR AND DENOMINATOR DECRYPTED ALONG WITH DURATION OF RATIO CALCULATION**

In [42]:
data = pd.read_csv('/content/sample_data/combined_financial_data_idx.csv')

In [43]:
import pandas as pd
import tenseal as ts
import time

# Set up the TenSEAL context
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.generate_galois_keys()
context.global_scale = 2**40
# Function to extract financial data for a company
def extract_financial_data(symbol):
    company_data = data[(data['symbol'] == symbol) & (data['2023'].notna())]
    return {
        'current_assets': float(company_data[company_data['account'] == 'Current Assets']['2023'].values[0]),
        'current_liabilities': float(company_data[company_data['account'] == 'Current Liabilities']['2023'].values[0]),
        'total_liabilities': float(company_data[company_data['account'] == 'Total Liabilities Net Minority Interest']['2023'].values[0]),
        'total_equity': float(company_data[company_data['account'] == 'Total Equity Gross Minority Interest']['2023'].values[0]),
        'total_assets': float(company_data[company_data['account'] == 'Total Assets']['2023'].values[0]),
        'net_income': float(company_data[company_data['account'] == 'Net Income']['2023'].values[0]),
        'total_revenue': float(company_data[company_data['account'] == 'Operating Revenue']['2023'].values[0]),
        'ebit': float(company_data[company_data['account'] == 'EBIT']['2023'].values[0]),
        'interest_expense': float(company_data[company_data['account'] == 'Interest Expense']['2023'].values[0]),
        'non_current_liabilities': float(company_data[company_data['account'] == 'Total Non Current Liabilities Net Minority Interest']['2023'].values[0])
    }

# List of companies
companies = ['AALI', 'ABMM', 'ACES', 'ACST']

# Extract data for all companies
company_data = {}
for company in companies:
    try:
        company_data[company] = extract_financial_data(company)
    except:
        print(f"Data not available for {company}")

# Function to extract and encrypt financial data for a company
def extract_and_encrypt_data(symbol):
    data = company_data[symbol]
    return {key: ts.ckks_vector(context, [value]) for key, value in data.items()}

# Extract and encrypt data for all companies
start_time = time.time()
encrypted_data = {company: extract_and_encrypt_data(company) for company in companies}
encryption_time = time.time() - start_time

# Helper function for division
def encrypted_division(numerator, denominator):
    num_value = numerator.decrypt()[0]
    den_value = denominator.decrypt()[0]
    result = num_value / den_value if den_value != 0 else float('inf')
    return ts.ckks_vector(context, [result])

# Function to calculate financial ratios
def calculate_ratios(data):
    calculation_times = {}

    start_time = time.time()
    current_ratio = encrypted_division(data['current_assets'], data['current_liabilities'])
    calculation_times['current_ratio'] = time.time() - start_time

    start_time = time.time()
    debt_to_equity = encrypted_division(data['total_liabilities'], data['total_equity'])
    calculation_times['debt_to_equity'] = time.time() - start_time

    start_time = time.time()
    roa = encrypted_division(data['net_income'], data['total_assets'])
    calculation_times['roa'] = time.time() - start_time

    start_time = time.time()
    profit_margin = encrypted_division(data['net_income'], data['total_revenue'])
    calculation_times['profit_margin'] = time.time() - start_time

    start_time = time.time()
    asset_turnover = encrypted_division(data['total_revenue'], data['total_assets'])
    calculation_times['asset_turnover'] = time.time() - start_time

    start_time = time.time()
    ebit_plus_interest = data['ebit'] + data['interest_expense']
    interest_coverage = encrypted_division(ebit_plus_interest, data['interest_expense'])
    calculation_times['interest_coverage'] = time.time() - start_time

    start_time = time.time()
    total_debt = data['current_liabilities'] + data['non_current_liabilities']
    debt_ratio = encrypted_division(total_debt, data['total_assets'])
    calculation_times['debt_ratio'] = time.time() - start_time

    return (current_ratio, debt_to_equity, roa, profit_margin, asset_turnover, interest_coverage, debt_ratio), calculation_times

# Calculate ratios for all companies
company_ratios = {}
calculation_times = {}
for company, data in encrypted_data.items():
    company_ratios[company], calculation_times[company] = calculate_ratios(data)

# Function to decrypt and print results
def decrypt_and_print_ratios(company_name, ratios):
    current_ratio, debt_to_equity, roa, profit_margin, asset_turnover, interest_coverage, debt_ratio = ratios
    print(f"\nFinancial Ratios for {company_name}:")
    print(f"Current Ratio: {current_ratio.decrypt()[0]:.4f}")
    print(f"Debt to Equity Ratio: {debt_to_equity.decrypt()[0]:.4f}")
    print(f"ROA: {roa.decrypt()[0]:.4f}")
    print(f"Profit Margin: {profit_margin.decrypt()[0]:.4f}")
    print(f"Asset Turnover: {asset_turnover.decrypt()[0]:.4f}")
    print(f"Interest Coverage: {interest_coverage.decrypt()[0]:.4f}")
    print(f"Debt Ratio: {debt_ratio.decrypt()[0]:.4f}")

# Print results for all companies
for company, ratios in company_ratios.items():
    decrypt_and_print_ratios(company, ratios)

# Function to print original ratios
def print_original_ratios(company_data):
    for company, data in company_data.items():
        current_ratio = data['current_assets'] / data['current_liabilities']
        debt_to_equity = data['total_liabilities'] / data['total_equity']
        roa = data['net_income'] / data['total_assets']
        profit_margin = data['net_income'] / data['total_revenue']
        asset_turnover = data['total_revenue'] / data['total_assets']
        interest_coverage = (data['ebit'] + data['interest_expense']) / data['interest_expense']
        debt_ratio = (data['current_liabilities'] + data['non_current_liabilities']) / data['total_assets']

        print(f"\nOriginal Financial Ratios for {company}:")
        print(f"Current Ratio: {current_ratio:.4f}")
        print(f"Debt-to-Equity Ratio: {debt_to_equity:.4f}")
        print(f"Return on Assets (ROA): {roa:.4f}")
        print(f"Profit Margin: {profit_margin:.4f}")
        print(f"Asset Turnover: {asset_turnover:.4f}")
        print(f"Interest Coverage: {interest_coverage:.4f}")
        print(f"Debt Ratio: {debt_ratio:.4f}")

# Print original ratios
print("\nOriginal Ratios:")
print_original_ratios(company_data)

# Print timing results
print("\nCalculation Times:")
for company, times in calculation_times.items():
    print(f"\n{company}:")
    for ratio, time_taken in times.items():
        print(f"  {ratio}: {time_taken:.6f} seconds")

print(f"\nEncryption Time: {encryption_time:.4f} seconds")



Financial Ratios for AALI:
Current Ratio: 1.8336
Debt to Equity Ratio: 0.2783
ROA: 0.0366
Profit Margin: 0.0509
Asset Turnover: 0.7192
Interest Coverage: 7.8662
Debt Ratio: 0.2177

Financial Ratios for ABMM:
Current Ratio: 0.9858
Debt to Equity Ratio: 1.8418
ROA: 0.1340
Profit Margin: 0.1936
Asset Turnover: 0.6923
Interest Coverage: 5.5011
Debt Ratio: 0.6481

Financial Ratios for ACES:
Current Ratio: 7.4151
Debt to Equity Ratio: 0.2533
ROA: 0.0985
Profit Margin: 0.1003
Asset Turnover: 0.9818
Interest Coverage: 16.1611
Debt Ratio: 0.2021

Financial Ratios for ACST:
Current Ratio: 0.9986
Debt to Equity Ratio: 5.5869
ROA: -0.1036
Profit Margin: -0.1150
Asset Turnover: 0.9007
Interest Coverage: -144.5726
Debt Ratio: 0.8482

Original Ratios:

Original Financial Ratios for AALI:
Current Ratio: 1.8336
Debt-to-Equity Ratio: 0.2783
Return on Assets (ROA): 0.0366
Profit Margin: 0.0509
Asset Turnover: 0.7192
Interest Coverage: 7.8662
Debt Ratio: 0.2177

Original Financial Ratios for ABMM:
Curren